In [3]:
import wandb
from keras.datasets import fashion_mnist
import numpy as np
import cli_args
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
# wandb.init (
# # set the wand project where this run will be logged
# project="CS6910_Assignment_1",
# )
# # Loading the fashion-MNIST dataset
# (x_train, y_train), (_test, _test) = fashion_mnist.load_data ()
# #class names for fashion-MIST
# class_names = ['T-shirt',
# 'Trouser',
# 'Pullover',
# 'Dress',
# 'Sandal', 'Shirt',
# 'Sneaker',
# 'Bag',
# 'Coat',
# 'Ankle boot']
# # creating 2x5 grid
# img={}
# for i in range(10):
#     # to find first image in the training set with class label i
#     idx = np.where (y_train == i)[0][0]
#     # Plot the image
#     img[class_names[i]] = wandb.Image(x_train[idx], caption=class_names[i])
# wandb.log(img)
# # [optional] finish the wand run, necessary in notebooks
# wandb.finish()

In [3]:
arguments = cli_args.argumentsIntake()

usage: ipykernel_launcher.py [-h] [-wp WANDB_PROJECT] [-we WANDB_ENTITY]
                             [-d DATASET] [-e EPOCHS] [-b BATCH_SIZE]
                             [-l LOSS] [-o OPTIMIZER] [-lr LEARNING_RATE]
                             [-m MOMENTUM] [-beta BETA] [-beta1 BETA1]
                             [-beta2 BETA2] [-eps EPSILON] [-w_d WEIGHT_DECAY]
                             [-w_i WEIGHT_INIT] [-nhl NUM_LAYERS]
                             [-sz HIDDEN_SIZE [HIDDEN_SIZE ...]]
                             [-a ACTIVATION] [-oa OUTPUT_ACTIVATION]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/vanditshah/Library/Jupyter/runtime/kernel-794a2b15-074b-4a45-bd73-f47f686c1e29.json


SystemExit: 2

/opt/homebrew/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
class Encoder:
    def __init__(self):
        pass
    def one_hot_encoder(self, num_of_classes):
        pass


In [4]:
class ActiviationFunction:
    def __init__(self, name):
        self.act_name = name
    
    @property
    def setname(self, new_name):
        self.act_name = new_name

    def activate(self, data, backprop = False):
        if self.act_name == "identity" and backprop:
            pass
        elif self.act_name == "identity" and not backprop:
            return self.identity(data)
        elif self.act_name == "sigmoid" and backprop:
            return self.backprop_sigmoid(data)
        elif self.act_name == "sigmoid" and not backprop:
            return self.sigmoid(data)
        elif self.act_name == "tanh" and backprop:
            return self.backprop_tanh(data)
        elif self.act_name == "tanh" and not backprop:
            return self.tanh(data)
        elif self.act_name == "ReLU" and backprop:
            pass
        elif self.act_name == "ReLU" and not backprop:
            return self.tanh(data)

    
    def identity(self, data):
        return data
    
    def backprop_identity(self, data):
        pass

    def sigmoid(self, data):
        return 1/(1 + np.exp(-data))
    
    def backprop_sigmoid(self, data):
        temp = self.sigmoid(data)
        return (1 - temp)*temp
    
    def relu(self, data):
        return np.max(0, data)
    
    def backprop_relu(self, data):
        return np.heaviside(data, 1)
    
    def tanh(self, data):
        return np.tanh(data)
    
    def backprop_tanh(self, data):
        return 1 - np.square(np.tanh(data))
    
    def softmax(self, data):
        _max = np.max(data)
        numerator = np.exp(data - _max)
        denominator = np.sum(numerator)
        return numerator/denominator

    def backprop_softmax(self, data):
        _softmax = self.softmax(data)
        return _softmax * (1 - _softmax)


In [5]:
class LossFunction:
    def __init__(self, name):
        self.name = name
    
    def getLoss(self, y, yhat):
        if self.name == "cross_entropy":
            return self.cross_entropy_loss(y, yhat)
        elif self.name == "mean_squared_error":
            return self.mean_squared_error_loss(y, yhat)
    
    def getGradient(self, y, yhat):
        if self.name == "cross_entropy":
            return self.backprop_cross_entropy(y, yhat)
        elif self.name == "mean_squared_error":
            return self.backprop_mean_squared_loss(y, yhat)
    
    def cross_entropy_loss(self, y, yhat):
        probability_predicted = yhat[np.argmax(y)]
        if probability_predicted <= 0:
            probability_predicted += 10**(-9)
        return -np.log(probability_predicted)
    
    def mean_squared_error_loss(self,y, yhat):
        return (1/2)*np.sum(np.square(yhat - y))
    
    def backprop_cross_entropy(self, y, yhat):
        return yhat - y
    
    def backprop_mean_squared_loss(self, y, yhat):
        return (yhat - y) * yhat * (1 - yhat)

In [1]:
class NeuralNetwork:
    def __init__(self, arguments) -> None:
        self.args_backup = arguments # is saved for later variable purposes
        self.n_hidden_layers = arguments.num_layers
        self.neurons_h_layers = arguments.hidden_size
        self.h_activation_func_name = arguments.activation
        self.output_activation_func_name = arguments.output_activation
        self.weight_decay = arguments.weight_decay
        self.epsilon = arguments.epsilon
        self.learning_rate = arguments.learning_rate
        self.optimizer_name = arguments.optimizer
        self.loss_func_name = arguments.loss
        self.batch_size = arguments.batch_size
        self.epochs = arguments.epochs
        self.dataset_name = arguments.dataset

        self.load_dataset()

        self.layers = [self.x_train.shape[1]] + [self.neurons_h_layers]*self.n_hidden_layers + [self.y_train[0].shape[0]]
        self.n_layers = self.n_hidden_layers + 2

        self.activation = ActiviationFunction(arguments.activation)
        self.outputActivation = ActiviationFunction(arguments.output_activation)
        self.loss = LossFunction(arguments.loss)

        self.init_parameters()

    def load_dataset(self):
        preprocessor = StandardScaler()
        if self.dataset_name == "fashion_mnist":
            from keras.datasets import fashion_mnist
            (self.x_train, self.y_train), (self.x_test, self.y_test) = fashion_mnist.load_data()
        elif self.dataset_name == "mnist":
            from keras.datasets import mnist
            (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()
        else:
            print("404 Dataset Not Found!")
            return

        self.x_train = self.x_train.astype('float64')
        self.y_train = self.y_train.astype('float64')
        self.x_test = self.x_test.astype('float64')
        self.y_test = self.y_test.astype('float64')

        self.x_train = self.x_train.reshape(self.x_train.shape[0],self.x_train.shape[1]*self.x_train.shape[2])
        self.x_train = preprocessor.fit_transform(self.x_train)
        self.y_train = self.y_train.reshape(self.y_train.shape[0],1)
        self.y_train = to_categorical(self.y_train)

        self.x_test = self.x_test.reshape(self.x_test.shape[0],self.x_test.shape[1]*self.x_test.shape[2])
        self.x_test = preprocessor.fit_transform(self.x_test)
        self.y_test = self.y_test.reshape(self.y_test.shape[0],1)
        self.y_test = to_categorical(self.y_test)

        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(self.x_train, self.y_train, test_size=0.10, random_state=42)
    
    def init_parameters(self, debug = False):
        if self.args_backup.weight_init == "random":
            self.randomInit(debug)
        elif self.args_backup.weight_init == "xavier":
            self.xavierInit(debug)
    
    def randomInit(self, _print=False):
        self.parameters = {}
        constant = 0.04
        for i in range(1, self.n_layers):
            self.parameters[f"W{i}"] = np.random.randn(self.layers[i], self.layers[i-1])*constant
            self.parameters[f"b{i}"] = np.zeros((self.layers[i], 1))
            if(_print):
                print(f'W{i} -> {self.parameters["W" + str(i)].shape}')
                print(f'b{i} -> {self.parameters["b" + str(i)].shape}')
    
    def xavierInit(self, _print=False):
        self.parameters = {}
        for i in range(1, self.n_layers):
            self.parameters[f"W{i}"] = np.random.randn(self.layers[i], self.layers[i - 1]) * np.sqrt(2/ (self.layers[i - 1] + self.layers[i]))
            self.parameters[f"b{i}"] = np.zeros((self.layers[i], 1))
            if(_print):
                print(f'W{i} -> {self.parameters["W" + str(i)].shape}')
                print(f'b{i} -> {self.parameters["b" + str(i)].shape}')

    
    def gradsInit(self, debug = False):
        temp_gradients = {}
        for i in range(1, self.n_layers):
            temp_gradients[f"W{i}"] = np.zeros((self.layers[i], self.layers[i - 1]))
            temp_gradients[f"b{i}"] = np.zeros((self.layers[i], 1))
            if debug:
                print(f'W{i} -> {temp_gradients["W" + str(i)].shape}')
                print(f'b{i} -> {temp_gradients["b" + str(i)].shape}')
        return temp_gradients

    
    def forward_propagation(self, data):
        self.a = {}  # preactivation
        self.h = {}  # activation
        self.h["h0"] = data.reshape(len(data), 1)
        
        for i in range(1, self.n_layers - 1):
            self.a[f"a{i}"] = np.matmul(self.parameters[f"W{i}"].T,self.h[f"h{i-1}"]) + self.parameters[f"b{i}"]
            self.h[f"h{i}"] = self.activation.activate(self.a[f"a{i}"])
        
        # for output layer
        self.a[f"a{self.n_layers-1}"] = np.matmul(self.parameters[f"W{self.n_layers-1}"].T,self.h[f"h{self.n_layers-2}"]) + self.parameters[f"b{self.n_layers-1}"]
        self.h[f"h{self.n_layers - 1}"] = self.outputActivation.activate(self.a[f"a{i}"])
    

    def back_propagation(self, data):
        gradients = {}
        gradients[f"a{self.n_layers - 1}"] = self.loss.getGradient(data.T, self.h[f"h{self.n_layers - 1}"])
        for i in range(self.n_layers - 1, 1, -1):
            gradients[f"W{i}"] = np.dot(gradients[f'a{i}'], self.h[f'h{i-1}'])
            gradients[[f"b{i}"]] = gradients[f"a{i}"]
            gradients[f"h{i-1}"] = np.dot(self.parameters[f"W{i}"].T, gradients[f"a{i}"])
            gradients[f"a{i-1}"] = gradients[f"h{i-1}"] * self.activation.activate(self.a[f"a{i-1}"], backprop=True)
        
        gradients[f"W{1}"] = np.dot(gradients[f'a{1}'], self.h[f'h{1-1}'])
        gradients[[f"b{1}"]] = gradients[f"a{1}"]
        gradients[f"h{1-1}"] = np.dot(self.parameters[f"W{1}"].T, gradients[f"a{1}"])

        return gradients

In [2]:
class Optimiser(NeuralNetwork):
    def __init__(self, arguments):
        super().__init__(arguments)
        self.parameters_without_activations = None
        self.separate_weights_and_biases()
    
    def update_parameters(self):
        for key in self.parameters_without_activations.keys():
            self.parameters[key] = self.parameters_without_activations[key]
    
    def separate_weights_and_biases(self):
        for key, value in self.parameters:
            if "W" in key or "b" in key:
                self.parameters_without_activations[key] = value
    
    def generateMetrics(self, x_data, y_data, _type = ""):
        predictions = []
        y = []
        yhat = []
        loss_history = []
        for x,yt in tqdm(zip(x_data , y_data), total=len(y_data), desc="Loss calculation", leave = False):
            self.h, self.a = self.forward_propagation(x)
            yhat = np.argmax(self.h['h' + str(self.L-1)])
            yt = yt.reshape(len(yt),1)
            _class = np.argmax(yt)
            y.append(_class)
            yhat.append(yhat)
            predictions.append(yhat == _class)
            loss_history.append(self.loss.getLoss(self.h['h' + str(self.L-1)],yt))

        accuracy = np.sum(predictions)/len(predictions)
        loss = np.sum(loss_history)/len(loss_history)

        print(f"{_type} Accuracy: {accuracy}", end = " ")
        print(f"{_type} Loss: {loss}", end = "\n")
			
        return yhat, y, accuracy*100, loss
    
    def stochastic_gradient_descent(self):
        _parameters = self.parameters_without_activations
        for i in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = Stochastic Gradient Descent", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] = gradients[key] + delta[key]
                
                if (id + 1) % self.batch_size == 0:
                    for key in _parameters.keys():
                        _parameters[key] = _parameters[key] - self.learning_rate * gradients[key]
                        gradients = self.gradsInit()
        
        
        self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
        self.generateMetrics(self.x_val, self.y_val, _type = "Train")
        self.generateMetrics(self.x_test, self.y_test, _type = "Test")

        self.update_parameters()
    
    def moment_based_gradient_descent(self):
        self.momentum = self.args_backup.momentum
        _parameters = self.parameters_without_activations
        _global = self.gradsInit()
        for i in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            lookahead = self.gradsInit()
            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = Momentum Based Gradient Descent", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] += delta[key]
                
                if (id + 1) % self.batch_size == 0:
                    for key in lookahead.keys():
                        lookahead[key] = self.momentum * _global[key] + self.learning_rate * gradients[key]
                    
                        _parameters[key] = _parameters[key] - lookahead[key]
                    
                        _global[key] = lookahead[key]
                    
                    gradients = self.gradsInit()
        
            self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
            self.generateMetrics(self.x_val, self.y_val, _type = "Train")
            self.generateMetrics(self.x_test, self.y_test, _type = "Test")
            
        self.update_parameters()
    
    def nestrov_gradient_descent(self):
        self.momentum = self.args_backup.momentum # beta
        _parameters = self.parameters_without_activations
        _global = self.gradsInit()
        for i in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            lookahead = self.gradsInit()

            # partial init
            for key in lookahead.keys():
                    lookahead[key] = self.momentum * lookahead[key]
            
            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = Nestrov Gradient Descent", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] += delta[key]

                
                if (id + 1) % self.batch_size == 0:
                    for key in lookahead.keys():
                        lookahead[key] = self.momentum * lookahead[key] + self.learning_rate * gradients[key]
                    
                        _parameters[key] = _parameters[key] - lookahead[key]
                    
                        _global[key] = lookahead[key]

                        gradients = self.gradsInit()

            self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
            self.generateMetrics(self.x_val, self.y_val, _type = "Train")
            self.generateMetrics(self.x_test, self.y_test, _type = "Test")

        self.update_parameters()
    
    def rms_prop(self):
        self.beta = self.args_backup.beta
        self.epsilon = self.args_backup.epsilon
        _parameters = self.parameters_without_activations
        _global = self.gradsInit()
        for i in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = RMS Prop", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] += delta[key]

                
                if (id + 1) % self.batch_size == 0:
                    for key in _global.keys():
                        _global[key] = self.beta * _global[key] + (1 - self.beta) * (gradients[key]**2)
                    
                        _parameters[key] = _parameters[key] - (self.learning_rate * gradients[key])/ (np.sqrt(_global[key] + self.epsilon))
                    
                    gradients = self.gradsInit()
        

            self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
            self.generateMetrics(self.x_val, self.y_val, _type = "Train")
            self.generateMetrics(self.x_test, self.y_test, _type = "Test")

        self.update_parameters()
    

    def adam(self):
        self.beta1 = self.args_backup.beta1
        self.beta2 = self.args_backup.beta2
        self.epsilon = self.args_backup.epsilon

        _parameters = self.parameters_without_activations

        m_gradients = self.gradsInit()
        v_gradients = self.gradsInit()

        m_gradients_hat = self.gradsInit()
        v_gradients_hat = self.gradsInit()

        for epoch in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = Adam", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] += delta[key]
                

                if (id+1) % self.batch_size == 0:
                    for key in m_gradients.keys():
                        m_gradients[key] = self.beta1 * m_gradients[key] + (1 - self.beta1) * gradients[key]
                    
                        v_gradients[key] = self.beta2 * v_gradients[key] + (1 - self.beta2) * (gradients[key]**2)

                    
                        m_gradients_hat[key] = m_gradients[key] / (1 - self.beta1 ** (epoch + 1))
                        v_gradients_hat[key] = v_gradients[key] / (1 - self.beta2 ** (epoch + 1))

                    
                    for key in _parameters.keys():
                        _parameters[key] = _parameters[key] - (self.learning_rate * m_gradients_hat[key]) / np.sqrt(v_gradients_hat[key] + self.epsilon)
                    
                    gradients = self.gradsInit()
                
            self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
            self.generateMetrics(self.x_val, self.y_val, _type = "Train")
            self.generateMetrics(self.x_test, self.y_test, _type = "Test")

        self.update_parameters()
    

    def nadam(self):
        self.beta = self.args_backup.momentum
        self.beta1 = self.args_backup.beta1
        self.beta2 = self.args_backup.beta2
        self.epsilon = self.args_backup.epsilon

        _parameters = self.parameters_without_activations
        _global = self.gradsInit()

        m_gradients = self.gradsInit()
        v_gradients = self.gradsInit()

        m_gradients_hat = self.gradsInit()
        v_gradients_hat = self.gradsInit()

        for epoch in tqdm(range(self.epochs), desc="Epochs"):
            gradients = self.gradsInit()
            lookahead = self.gradsInit()

            # partial init
            for key in lookahead.keys():
                lookahead[key] = self.beta * lookahead[key]

            for id, x, y in tqdm(enumerate(zip(self.x_train, self.y_train)), desc = "Optimizer class = nAdam", leave = False):
                self.forward_propagation(x)
                delta = self.back_propagation(y)
                for key in gradients.keys():
                    gradients[key] += delta[key]
                

                if (id+1) % self.batch_size == 0:
                    for key in lookahead.keys():
                        lookahead[key] = self.beta * _global[key] + self.learning_rate * gradients[key]
                        m_gradients[key] = self.beta1 * m_gradients[key] + (1 - self.beta1) * gradients[key]
                    
                        v_gradients[key] = self.beta2 * v_gradients[key] + (1 - self.beta2) * (gradients[key]**2)
                    
                        m_gradients_hat[key] = m_gradients[key] / (1 - self.beta1 ** (epoch + 1))
                        v_gradients_hat[key] = v_gradients[key] / (1 - self.beta2 ** (epoch + 1))

                        _parameters[key] = _parameters[key] - (self.learning_rate * m_gradients_hat[key]) / np.sqrt(v_gradients_hat + self.epsilon)
                    
                    for key in _global.keys():
                        _global[key] = lookahead[key]

                    
                    gradients = self.gradsInit()
            
            self.generateMetrics(self.x_val, self.y_val, _type = "Validation")
            self.generateMetrics(self.x_val, self.y_val, _type = "Train")
            self.generateMetrics(self.x_test, self.y_test, _type = "Test")

        self.update_parameters()
    

    